In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as sp
import time

from helpers import load_data
from utils import split_data
from cross_validation import cross_validation_step_sgd

%matplotlib inline
%load_ext autoreload
%autoreload 2

# Load and preprocess data

In [2]:
path_dataset = "../data/data_train.csv"
ratings = load_data(path_dataset)

In [3]:
nF, nU = ratings.shape
print(nF, nU)

10000 1000


In [4]:
viewings = np.zeros((nF, nU))
nnz_rows, nnz_cols = ratings.nonzero()
for f, u in list(zip(nnz_rows, nnz_cols)):
    viewings[f, u] = 1

In [5]:
ratings = np.asarray(ratings.toarray())

In [6]:
user_means = np.sum(ratings, axis=0) / np.sum(viewings, axis=0)
user_stdDevs = (np.sum(ratings * ratings, axis=0) / np.sum(viewings, axis=0)) - user_means * user_means

In [7]:
normalized_ratings = viewings * (ratings - user_means) / user_stdDevs

# Build graph

In [118]:
similarities = np.zeros((nU, nU))
user_commonViewings = np.zeros((nU, nU))
for f in range(nF):
    if(f % 100 == 0):
        print(f)
    users = np.where(viewings[f,:] == 1)[0]
    nRatings = len(users)
    for i in range(nRatings):
        for j in range(i + 1, nRatings):
            similarities[users[i], users[j]] += normalized_ratings[f, users[i]] * normalized_ratings[f, users[j]]
            similarities[users[j], users[i]] = similarities[users[i], users[j]]
            user_commonViewings[users[i], users[j]] += 1
            user_commonViewings[users[j], users[i]] += 1

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


In [151]:
def sim_to_dist(x):
    if(x <= 709):
        return 1./(1. + np.exp(x))
    return np.exp(-x)/(1. + np.exp(-x))

In [162]:
distances = np.zeros((nU, nU))
nnz_rows, nnz_cols = user_commonViewings.nonzero()
for ui, uj in list(zip(nnz_rows, nnz_cols)):
    distances[ui, uj] = sim_to_dist(similarities[ui, uj])

In [205]:
neighbors = []
for ui in range(nU):
    n = []
    for uj in range(nU):
        if user_commonViewings[ui, uj] > 0:
            n.append(uj)
    neighbors.append(n)

# Shortest paths from one user vertex

In [193]:
def get_shortest_paths(user):
    dist_users = -1 * np.ones(nU)
    dist_users[user] = 0
    Q = {user}
    P = {1}
    P.remove(1)
    while len(Q) > 0:
        minDist = 1000000000000
        for u in Q:
            if dist_users[u] < minDist:
                uMin = u
        Q.remove(uMin)
        P.add(uMin)
        for u in neighbors[uMin]:
            if not u in P:
                if dist_users[u] == -1:
                    dist_users[u] = dist_users[uMin] + distances[uMin, u]
                else:
                    dist_users[u] = min(dist_users[u], dist_users[uMin] + distances[uMin, u])
                Q.add(u)
    print(len(P))
    return dist_users
#Problem : the graph is almost complete, and the distance does not satisfy triangular inequality 
# -> length of shortest path is always almost 0
# Fix : assume that in most of the cases the one-ring neighbors are enough, and use k-nn
# Or : be more restrictive on links (enough films in common)

# k nearest neighbors

In [209]:
sorted_neighbors = []
for ui in range(nU):
    n = [(uj, similarities[ui, uj]) for uj in neighbors[ui]]
    n = sorted(n, key=lambda x : x[1], reverse=1)
    sorted_neighbors.append(n)

In [238]:
def prediction(f, u, k):
    count = 0
    ref_neighbors = []
    n_neighbors = len(sorted_neighbors[u])
    while len(ref_neighbors) < k and count < n_neighbors:
        n = sorted_neighbors[u][count][0]
        if viewings[f, n] == 1 and similarities[u, n] > 0:
            ref_neighbors.append(n)
        count += 1
    if len(ref_neighbors) == 0:
        print("No (correlated) neighbors have seen the film")
        return user_means[u]
    meanGrade = 0
    totWeights = 0
    for n in ref_neighbors:
        meanGrade += (user_means[u] + user_stdDevs[u]*normalized_ratings[f, n]) * similarities[u,n]
        totWeights += similarities[u,n]
    meanGrade /= totWeights
    meanGrade = round(meanGrade)
    meanGrade = min(meanGrade, 5)
    meanGrade = max(meanGrade, 1)
    return meanGrade

In [244]:
kMax = 10
for k in range(1, kMax + 1):
    print('k={}:'.format(k))
    count = 0
    mse = 0
    for u in range(nU):
        if(u % 100 == 0):
            print('User #{}'.format(u + 1))
        for f in range(nF):
            if viewings[f, u] == 1:
                mse += (ratings[f, u] - prediction(f, u, k))**2
                count += 1
    print("RMSE : ")
    print(np.sqrt(mse / count))
            

k=1:
User #1
User #101
User #201
User #301
User #401
User #501
User #601
User #701
User #801
No (correlated) neighbors have seen the film
User #901
RMSE : 
1.40866848094
k=2:
User #1
User #101
User #201
User #301
User #401
User #501
User #601
User #701
User #801
No (correlated) neighbors have seen the film
User #901
RMSE : 
1.2393353954
k=3:
User #1
User #101
User #201
User #301
User #401
User #501
User #601
User #701
User #801
No (correlated) neighbors have seen the film
User #901
RMSE : 
1.16014810426
k=4:
User #1
User #101
User #201
User #301
User #401
User #501
User #601
User #701
User #801
No (correlated) neighbors have seen the film
User #901
RMSE : 
1.11650355277
k=5:
User #1
User #101
User #201
User #301
User #401
User #501
User #601
User #701
User #801
No (correlated) neighbors have seen the film
User #901
RMSE : 
1.08889620606
k=6:
User #1
User #101
User #201
User #301
User #401
User #501
User #601
User #701
User #801
No (correlated) neighbors have seen the film
User #901
RMS

In [245]:
for k in range(11, 21):
    print('k={}:'.format(k))
    count = 0
    mse = 0
    for u in range(nU):
        if(u % 100 == 0):
            print('User #{}'.format(u + 1))
        for f in range(nF):
            if viewings[f, u] == 1:
                mse += (ratings[f, u] - prediction(f, u, k))**2
                count += 1
    print("RMSE : ")
    print(np.sqrt(mse / count))

k=11:
User #1
User #101
User #201
User #301
User #401
User #501
User #601
User #701
User #801
No (correlated) neighbors have seen the film
User #901
RMSE : 
1.02501195514
k=12:
User #1
User #101
User #201
User #301
User #401
User #501
User #601
User #701
User #801
No (correlated) neighbors have seen the film
User #901
RMSE : 
1.02079194301
k=13:
User #1
User #101
User #201
User #301
User #401
User #501
User #601
User #701
User #801
No (correlated) neighbors have seen the film
User #901
RMSE : 
1.01757150957
k=14:
User #1
User #101
User #201
User #301
User #401
User #501
User #601
User #701
User #801
No (correlated) neighbors have seen the film
User #901
RMSE : 
1.01395713998
k=15:
User #1
User #101
User #201
User #301
User #401
User #501
User #601
User #701
User #801
No (correlated) neighbors have seen the film
User #901
RMSE : 
1.01154682515
k=16:
User #1
User #101
User #201
User #301
User #401
User #501
User #601
User #701
User #801
No (correlated) neighbors have seen the film
User #

In [246]:
ks = [10, 11, nU]
for k in ks:
    print('k={}:'.format(k))
    count = 0
    mse = 0
    for u in range(nU):
        if(u % 100 == 0):
            print('User #{}'.format(u + 1))
        for f in range(nF):
            if viewings[f, u] == 1:
                mse += (ratings[f, u] - prediction(f, u, k))**2
                count += 1
    print("RMSE : ")
    print(np.sqrt(mse / count))

k=10:
User #1
User #101
User #201
User #301
User #401
User #501
User #601
User #701
User #801
No (correlated) neighbors have seen the film
User #901
RMSE : 
1.0301879223
k=11:
User #1
User #101
User #201
User #301
User #401
User #501
User #601
User #701
User #801
No (correlated) neighbors have seen the film
User #901
RMSE : 
1.02501195514
k=1000:
User #1
User #101
User #201
User #301
User #401
User #501
User #601
User #701
User #801
No (correlated) neighbors have seen the film
User #901
RMSE : 
1.00205237017
